In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib

In [2]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
from urllib import urlencode

In [3]:
import rauth

In [23]:
#API_KEY = 'ae90aOVE_KXd0bPY8aSNPV0LvUdhfHfMuzSC8G0UH_ONoBGaLYOPSB1xZ8h9au9dp3reYFaca5N8G49gjll4FfY1CVYkBZv7A7Xkcg51KB7ahay7ADpmJAOz-LcFXHYx'

In [18]:
API_KEY = 'DZStxM87nhXlJBL3JsvXK9KjzxLiuPktRXFX_cm4_debQ4j5W_lbGL_uaWDCYW2nzvGG1MpPN5lUUx03mGaVRWlOqhsuaFFf_AO1IgKStDtJYbwPK4pzFBbRNxL6W3Yx'

In [5]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
EVENTS_PATH = '/v3/events'

In [7]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [22]:
def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [9]:
def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

In [10]:
def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [11]:
# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 50

In [12]:
def search_event(start_date,end_date,latitude,longitude,categories):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'start_date': start_date,
        'end_date': end_date,
        'latitude': latitude,
        'longitude': longitude,
        'categories': categories,
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, EVENTS_PATH, API_KEY, url_params=url_params)

In [13]:
from datetime import datetime, date
def getUnix(dt):
    return int((dt - datetime(1970,1,1)).total_seconds())

In [14]:
start_date = getUnix( datetime(2018,11,25) )
end_date = getUnix( datetime(2018,11,26) )
latitude = 40.70544486444615
longitude = -73.99429321289062
categories = 'nightlife' 

In [19]:
data = search_event(start_date,end_date,latitude,longitude,categories)

Querying https://api.yelp.com/v3/events ...


In [20]:
data.keys()

[u'total', u'events']

In [17]:
data['error']

{u'code': u'NOT_IN_DEVELOPER_BETA',
 u'description': u'This endpoint requires you be in our developer beta. To join, visit your "Manage App" page and click the join beta button.'}

In [95]:
data['events'][0].keys()

[u'category',
 u'business_id',
 u'event_site_url',
 u'attending_count',
 u'time_start',
 u'description',
 u'location',
 u'cost_max',
 u'time_end',
 u'is_free',
 u'name',
 u'interested_count',
 u'cost',
 u'image_url',
 u'longitude',
 u'latitude',
 u'is_official',
 u'id',
 u'tickets_url',
 u'is_canceled']

In [97]:
[d['name'] for d in data['events']]

[u'Rock the Yacht: Friday Night Party Cruise',
 u'Rock the Yacht: Saturday Night Party Cruise',
 u"Have You Brunch'd Lately? Brunch and Day Party Thanksgiving Weekend"]

In [101]:
[d['cost'] for d in data['events']]

[35.0, 35.0, None]

In [ ]:
#search(api_key, term, location)

In [24]:
query_api('night club', 'New York')

Querying https://api.yelp.com/v3/businesses/search ...
50 businesses found, querying business info for the top result "4nZlMrDw-mrvmfEVT-cQrw" ...
Querying https://api.yelp.com/v3/businesses/4nZlMrDw-mrvmfEVT-cQrw ...
Result for business "4nZlMrDw-mrvmfEVT-cQrw" found:
{ u'alias': u'the-village-underground-new-york',
  u'categories': [ { u'alias': u'danceclubs', u'title': u'Dance Clubs'},
                   { u'alias': u'lounges', u'title': u'Lounges'},
                   { u'alias': u'comedyclubs', u'title': u'Comedy Clubs'}],
  u'coordinates': { u'latitude': 40.7306935670033,
                    u'longitude': -74.0009226385494},
  u'display_phone': u'(212) 777-7745',
  u'hours': [ { u'hours_type': u'REGULAR',
                u'is_open_now': True,
                u'open': [ { u'day': 0,
                             u'end': u'0400',
                             u'is_overnight': True,
                             u'start': u'2100'},
                           { u'day': 1,
              

In [23]:
data = search(API_KEY, 'nightlife', 'new york')

Querying https://api.yelp.com/v3/businesses/search ...


In [27]:
data.keys()

[u'region', u'total', u'businesses']

In [33]:
data['businesses'][0].keys()

[u'rating',
 u'review_count',
 u'name',
 u'transactions',
 u'url',
 u'price',
 u'distance',
 u'coordinates',
 u'alias',
 u'image_url',
 u'categories',
 u'display_phone',
 u'phone',
 u'id',
 u'is_closed',
 u'location']

In [46]:
[b['name'] for b in data['businesses']]


[u'Rooftop 93',
 u'Playboy Club NYC',
 u'River Lounge',
 u'The Village Underground',
 u'Manina',
 u'Red Rabbit Club',
 u'Anytime Karaoke',
 u'Saint Julivert',
 u'Mexicue',
 u'Boca Restaurant and Lounge',
 u'RIO Kitchen & Wine',
 u'101 Brooklyn',
 u'Bar314',
 u'A+ Roof Bar',
 u'Pier 17',
 u'Keybar',
 u"Ada's Place",
 u'The Living Room',
 u'Club Cumming',
 u'New York Comedy Club',
 u'Basement',
 u'Savor Por Favor',
 u'Ascent Lounge',
 u'Harlem Nights Bar',
 u'Our Wicked Lady',
 u'El Cacto',
 u'Boobie Trap',
 u'Minami Lounge',
 u'Mailroom',
 u'Dkalabash',
 u'The Ship',
 u'RFTP',
 u'The Wilson',
 u'Fusia',
 u'The 75 Club',
 u'Rumpus Room',
 u'Skyline Bar',
 u'Nittis',
 u'Iron Horse NYC',
 u"Queen's Cabinet Bar & Lounge",
 u'On Now Lounge',
 u"e's BAR",
 u'Post Bar',
 u'C Lounge Restaurant & Hookah Bar',
 u'Turntable Caf\xe9 & Lounge',
 u'Luv Story Bar',
 u'Raines Law Room',
 u'Intersect by Lexus - NYC',
 u'Anotheroom',
 u'Bembe']

In [44]:
data['businesses'][0]['price']

u'$$'

In [42]:
data['region']

{u'center': {u'latitude': 40.70544486444615, u'longitude': -73.99429321289062}}

In [45]:
## radius parameter : The max value is 40000 meters (about 25 miles).